### Importación de Librerías Necesarias

La **Celda 2** importa todas las librerías esenciales para el proyecto de reconocimiento de emociones:

#### Librerías de Análisis de Datos
- **numpy**: Operaciones numéricas y manipulación de arrays
- **matplotlib.pyplot**: Visualización de gráficos
- **seaborn**: Mejora de visualizaciones con temas y estilos

#### Librerías de Machine Learning
- **tensorflow**: Framework de deep learning de Google
- **keras**: API de alto nivel dentro de TensorFlow para construir modelos de redes neuronales
  - layers: Capas para construir la arquitectura del modelo
  - ImageDataGenerator: Generación y augmentación de imágenes
  - callbacks: EarlyStopping, ReduceLROnPlateau, ModelCheckpoint para entrenamiento avanzado

#### Librerías de Visión por Computadora
- **cv2 (OpenCV)**: Captura de video, detección de rostros, procesamiento de imágenes
- **load_model**: Carga de modelos pre-entrenados

Estas librerías son el **fundamento** para todo el sistema: desde el entrenamiento del modelo CNN hasta la detección en tiempo real con la cámara web.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2
from tensorflow.keras.models import load_model

### Detección de Emociones en Tiempo Real (Versión Básica)

La **Celda 4** implementa un sistema básico de detección de emociones usando la cámara web. Funciona en un hilo separado para no bloquear la interfaz.

#### Carga del Modelo
- Carga el modelo CNN pre-entrenado desde el archivo `best_emotion_model.h5`
- Detecta 7 emociones: angry, disgust, fear, happy, neutral, sad, surprise

#### Gestión de Assets
- Carga imágenes PNG para cada emoción (en la carpeta Assets/)
- Mapea cada emoción a un color BGR específico (formato OpenCV)
- Asigna colores para visualización: rojo (cabreado), naranja (asqueado), azul (asustado), etc.

#### Funciones de Visualización
- `overlay_png_on_frame()`: Superpone imágenes PNG sobre rostros detectados con soporte para transparencia (canal alfa)
- `add_emotion_effects()`: Agrega efectos visuales según la emoción (bordes de color, círculos decorativos, texto)

#### Loop Principal de Detección
- Captura video de la webcam en resolución 640x480
- Convierte cada frame a escala de grises para detectar rostros
- Para cada rostro detectado:
  - Extrae la región y la redimensiona a 48x48 píxeles (formato del modelo)
  - Normaliza la imagen y realiza predicción con el modelo CNN
  - Aplica efectos visuales según la emoción detectada

#### Control
- Presionar 'q' para salir del programa
- Timeout de 5 minutos si no se cierra manualmente

In [ ]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import threading
import time
import os

model = load_model('best_emotion_model.h5')

emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
emotions_spanish = ['Cabreado', 'Asqueado', 'Asustado', 'Feliz', 'Neutral', 'Triste', 'Sorprendido']

png_files = {
    0: 'Assets/angry.jpg',
    1: 'Assets/disgust.png',
    2: 'Assets/fear.png',
    3: 'Assets/happy.png',
    4: 'Assets/neutral.png',
    5: 'Assets/sadness.png',
    6: 'Assets/surprise.png'
}

png_images = {}
for idx, fname in png_files.items():
    if os.path.exists(fname):
        img = cv2.imread(fname, cv2.IMREAD_UNCHANGED)
        if img is None:
            print(f"Advertencia: no se pudo leer {fname}")
        else:
            png_images[idx] = img
    else:
        print(f"Advertencia: archivo PNG no encontrado: {fname}")

emotion_config = {
    0: {'color': (0, 0, 255), 'name': 'Cabreado'},
    1: {'color': (0, 165, 255), 'name': 'Asqueado'},
    2: {'color': (255, 0, 0), 'name': 'Asustado'},
    3: {'color': (0, 255, 0), 'name': 'Feliz'},
    4: {'color': (128, 128, 128), 'name': 'Neutral'},
    5: {'color': (255, 0, 255), 'name': 'Triste'},
    6: {'color': (0, 255, 255), 'name': 'Sorprendido'}
}

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

stop_event = threading.Event()


def overlay_png_on_frame(frame, png, x, y, w, h, scale=1.0):
    if png is None:
        return frame

    target_w = int(w * scale)
    target_h = int(h * scale)

    png_h, png_w = png.shape[:2]
    if png_w == 0 or png_h == 0:
        return frame

    resized = cv2.resize(png, (target_w, target_h), interpolation=cv2.INTER_AREA)

    cx = x + w // 2
    cy = y + h // 2
    x1 = int(cx - target_w // 2)
    y1 = int(cy - target_h // 2)
    x2 = x1 + target_w
    y2 = y1 + target_h

    fh, fw = frame.shape[:2]
    ox1, oy1 = max(0, x1), max(0, y1)
    ox2, oy2 = min(fw, x2), min(fh, y2)

    if ox1 >= ox2 or oy1 >= oy2:
        return frame

    rx1 = ox1 - x1
    ry1 = oy1 - y1
    rx2 = rx1 + (ox2 - ox1)
    ry2 = ry1 + (oy2 - oy1)

    roi_frame = frame[oy1:oy2, ox1:ox2]
    roi_png = resized[ry1:ry2, rx1:rx2]

    if roi_png.shape[2] == 4:
        b, g, r, a = cv2.split(roi_png)
        alpha = a.astype(float) / 255.0
        alpha = cv2.merge([alpha, alpha, alpha])
        foreground = cv2.merge([b, g, r]).astype(float)
        background = roi_frame.astype(float)
        blended = cv2.convertScaleAbs(foreground * alpha + background * (1 - alpha))
    else:
        blended = roi_png[:, :, :3]

    frame[oy1:oy2, ox1:ox2] = blended
    return frame


def add_emotion_effects(frame, emotion_idx, x, y, w, h, confidence):
    config = emotion_config.get(emotion_idx, {'color': (255, 255, 255), 'name': str(emotion_idx)})

    png = png_images.get(emotion_idx)
    if png is not None:
        scale = 1.2
        frame = overlay_png_on_frame(frame, png, x, y, w, h, scale=scale)

        text = f"{config['name']}: {confidence:.2%}"
        cv2.putText(frame, text, (x + 5, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        return frame

    color = config['color']

    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 3)

    overlay = frame.copy()
    cv2.rectangle(overlay, (x - 5, y - 50), (x + 200, y - 10), color, -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)

    text = f"{config['name']}: {confidence:.2%}"
    cv2.putText(frame, text, (x + 5, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.circle(frame, (x + w//2, y + h//2), w//2 + 10, color, 2)

    for corner_x, corner_y in [(x, y), (x+w, y), (x, y+h), (x+w, y+h)]:
        cv2.circle(frame, (corner_x, corner_y), 5, color, -1)

    return frame


def emotion_detection_thread():
    cap = cv2.VideoCapture(0)

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    print("Ventana abierta. Presiona 'q' para salir.")

    while not stop_event.is_set():
        ret, frame = cap.read()

        if not ret:
            print("Error al capturar la cámara")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.3,
            minNeighbors=5,
            minSize=(30, 30)
        )

        emotion_detected = None
        max_confidence = 0

        for (x, y, w, h) in faces:
            roi_gray = gray[y:y + h, x:x + w]

            roi_resized = cv2.resize(roi_gray, (48, 48))

            roi_normalized = roi_resized / 255.0

            roi_input = np.expand_dims(roi_normalized, axis=-1)
            roi_input = np.expand_dims(roi_input, axis=0)

            prediction = model.predict(roi_input, verbose=0)
            emotion_idx = np.argmax(prediction)
            confidence = prediction[0][emotion_idx]

            if confidence > max_confidence:
                max_confidence = confidence
                emotion_detected = emotion_idx

            frame = add_emotion_effects(frame, emotion_idx, x, y, w, h, confidence)

        if emotion_detected is not None:
            config = emotion_config[emotion_detected]
            info_text = f"Emocion Principal: {config['name']}"
            cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, config['color'], 2)

            overlay = frame.copy()
            color = config['color']
            cv2.rectangle(overlay, (0, 0), (frame.shape[1], 60), color, -1)
            cv2.addWeighted(overlay, 0.1, frame, 0.9, 0, frame)

        cv2.imshow('Deteccion de Emociones', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_event.set()
            break

    cap.release()
    cv2.destroyAllWindows()

thread = threading.Thread(target=emotion_detection_thread, daemon=True)
thread.start()

thread.join(timeout=300)

Ventana abierta. Presiona 'q' para salir.
Las emociones cambiarán el color y los efectos visuales

Deteccion finalizada
Sistema de detección de emociones finalizado
Deteccion finalizada
Sistema de detección de emociones finalizado


### Detección de Emociones en Tiempo Real (Versión Avanzada con Multimedia)

La **Celda 5** implementa el sistema completo de reconocimiento de emociones con feedback multimedia, máquina de estados y control de alarmas.

#### Importaciones y Configuración Inicial
- Importa simpleaudio para reproducción de audio WAV con control de bucles
- Carga el modelo CNN pre-entrenado
- Define configuración de colores y nombres de emociones en español
- Carga Haar Cascade para detección de rostros

#### Búsqueda de Assets
- `find_file()`: Función recursiva que busca archivos de audio e imágenes en el árbol de directorios
- Busca archivos de audio: `SmokeAlarmChirp.wav` (sonido aleatorio), `AngerAlarm.wav` (alarma de cabreado)
- Busca imagen: `DangerTriangle.png` (triángulo de advertencia, redimensionado a 200x200)

#### Funciones de Audio
- `play_random_smoke_alarm()`: Reproduce sonido aleatorio en hilo separado cada 5-10 segundos
- `play_anger_alarm()`: Reproduce la alarma en bucle usando simpleaudio mientras `anger_alarm_playing` sea True
- `stop_anger_alarm_now()`: Detiene inmediatamente la reproducción de audio llamando al método `.stop()`

#### Visualización Multimedia
- `overlay_danger_triangle()`: Superpone el triángulo centrado con soporte para transparencia (canal alfa RGBA)
- Triángulo parpadea cada 0.5 segundos cuando está activo el estado de cabreado

#### Máquina de Estados - Lógica Principal
La detección funciona con dos estados principales:

**Estado Normal**:
- Detecta emociones y aplica overlay de color a toda la pantalla (20% opacidad)
- Muestra nombre y confianza de la emoción detectada
- Reproduce sonidos aleatorios cada 5-10 segundos

**Estado Cabreado (angry)**:
- Activado cuando se detecta emoción 0 (cabreado)
- Inicia temporizador de 5 segundos
- Cubre pantalla con overlay rojo (40% opacidad)
- Muestra triángulo de peligro parpadeante
- Reproduce alarma en bucle continuo
- Decrementa contador visual cada segundo

#### Transiciones de Estado
- **Normal → Cabreado**: Cuando `emotion_detected == 0`
- **Cabreado → Normal**: Si se detecta feliz (emoción 3) dentro de 5 segundos, se detiene alarma y se reinicia
- **Cabreado → Fin**: Si pasan 5 segundos sin sonreír, se detiene alarma y cierra aplicación

#### Variables Globales de Control
- `anger_alarm_playing`: Flag booleano para controlar bucle de alarma
- `anger_alarm_play_obj`: Referencia al objeto de reproducción para detención inmediata
- `cabreado_active`: Indica si el sistema está en estado de cabreado
- `danger_triangle_visible`: Controla visibilidad parpadeante del triángulo

#### Thread Principal
- `emotion_detection_thread()`: Ejecuta en hilo daemon la lógica principal
- Captura video 640x480, procesa frames en tiempo real
- Mantiene bucle activo hasta presionar 'q' o timeout de 5 segundos en cabreado
- Limpia recursos: libera cámara, cierra ventanas, detiene alarma


In [ ]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import threading
import time
import random
import os
import simpleaudio as sa

model = load_model('best_emotion_model.h5')

emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

emotion_config = {
    0: {'color': (0, 0, 255), 'name': 'Cabreado'},
    1: {'color': (0, 165, 255), 'name': 'Asqueado'},
    2: {'color': (255, 0, 0), 'name': 'Asustado'},
    3: {'color': (0, 255, 0), 'name': 'Feliz'},
    4: {'color': (128, 128, 128), 'name': 'Neutral'},
    5: {'color': (255, 0, 255), 'name': 'Triste'},
    6: {'color': (0, 255, 255), 'name': 'Sorprendido'}
}

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
stop_event = threading.Event()

def find_file(filename):
    if os.path.exists(filename):
        return os.path.abspath(filename)
    for root, dirs, files in os.walk('.'):
        if filename in files:
            return os.path.abspath(os.path.join(root, filename))
    return None

smoke_alarm_path = find_file('Assets/SmokeAlarmChirp.wav')
anger_alarm_path = find_file('Assets/AngerAlarm.wav')

danger_triangle_path = find_file('Assets/DangerTriangle.png')
danger_triangle = None

if danger_triangle_path:
    danger_triangle = cv2.imread(danger_triangle_path, cv2.IMREAD_UNCHANGED)
    if danger_triangle is not None:
        danger_triangle = cv2.resize(danger_triangle, (200, 200), interpolation=cv2.INTER_AREA)
    else:
        print("Error cargando DangerTriangle.png")
else:
    print("DangerTriangle.png NO encontrado")

last_random_sound_time = 0
anger_alarm_playing = False
danger_triangle_visible = False
danger_triangle_timer = 0
anger_alarm_play_obj = None


def play_random_smoke_alarm():
    if not smoke_alarm_path:
        return

    def run():
        try:
            sound = sa.WaveObject.from_wave_file(smoke_alarm_path)
            sound.play()
        except Exception as e:
            print(f"Error reproduciendo SmokeAlarmChirp.wav: {e}")

    threading.Thread(target=run, daemon=True).start()


def play_anger_alarm():
    global anger_alarm_play_obj, anger_alarm_playing

    if not anger_alarm_path:
        return

    def loop():
        global anger_alarm_play_obj
        sound = sa.WaveObject.from_wave_file(anger_alarm_path)

        while anger_alarm_playing and not stop_event.is_set():
            anger_alarm_play_obj = sound.play()

            while anger_alarm_play_obj.is_playing():
                if not anger_alarm_playing or stop_event.is_set():
                    break
                time.sleep(0.05)

    threading.Thread(target=loop, daemon=True).start()


def stop_anger_alarm_now():
    global anger_alarm_play_obj
    try:
        if anger_alarm_play_obj is not None:
            anger_alarm_play_obj.stop()
    except:
        pass


def overlay_danger_triangle(frame, danger_triangle):
    if danger_triangle is None:
        return frame
    
    h, w = frame.shape[:2]
    tri_h, tri_w = danger_triangle.shape[:2]
    x = (w - tri_w) // 2
    y = (h - tri_h) // 2
    roi = frame[y:y + tri_h, x:x + tri_w]

    if danger_triangle.shape[2] == 4:
        b, g, r, a = cv2.split(danger_triangle)
        alpha = a.astype(float) / 255.0
        alpha = cv2.merge([alpha, alpha, alpha])
        blended = cv2.convertScaleAbs(cv2.merge([b, g, r]).astype(float) * alpha + roi.astype(float) * (1 - alpha))
        frame[y:y + tri_h, x:x + tri_w] = blended
    else:
        frame[y:y + tri_h, x:x + tri_w] = danger_triangle[:, :, :3]

    return frame


def emotion_detection_thread():
    global last_random_sound_time, anger_alarm_playing, danger_triangle_visible, danger_triangle_timer

    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)

    cabreado_timer = None
    cabreado_active = False

    print("\nVentana abierta. Pulsa 'q' para salir.\n")

    while not stop_event.is_set():
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(30, 30))

        emotion_detected = None
        max_confidence = 0

        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_resized = cv2.resize(roi_gray, (48, 48))
            roi_input = np.expand_dims(np.expand_dims(roi_resized / 255.0, -1), 0)

            pred = model.predict(roi_input, verbose=0)
            idx = np.argmax(pred)
            conf = pred[0][idx]

            if conf > max_confidence:
                max_confidence = conf
                emotion_detected = idx

        if current_time - last_random_sound_time > random.uniform(5, 10):
            if random.random() > 0.7:
                play_random_smoke_alarm()
            last_random_sound_time = current_time

        if cabreado_active:

            overlay = frame.copy()
            overlay[:] = np.array(emotion_config[0]['color'], np.uint8)
            cv2.addWeighted(overlay, 0.4, frame, 0.6, 0, frame)

            if current_time - danger_triangle_timer > 0.5:
                danger_triangle_visible = not danger_triangle_visible
                danger_triangle_timer = current_time
            
            if danger_triangle_visible:
                frame = overlay_danger_triangle(frame, danger_triangle)

            remaining = 5 - (current_time - cabreado_timer)
            if remaining > 0:
                cv2.putText(frame, f"Mosqueo detectado! Te quedan {int(remaining)} seg",
                            (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

                if emotion_detected == 3:
                    cabreado_active = False
                    anger_alarm_playing = False
                    stop_anger_alarm_now()

            else:
                anger_alarm_playing = False
                stop_anger_alarm_now()
                stop_event.set()

        else:
            if emotion_detected == 0:
                cabreado_active = True
                cabreado_timer = current_time
                danger_triangle_timer = current_time
                danger_triangle_visible = True
                if not anger_alarm_playing:
                    anger_alarm_playing = True
                    play_anger_alarm()

            else:
                if emotion_detected is not None:
                    color = np.array(emotion_config[emotion_detected]['color'], np.uint8)
                    overlay = frame.copy()
                    overlay[:] = color
                    cv2.addWeighted(overlay, 0.2, frame, 0.8, 0, frame)

                    cv2.putText(frame,
                                f"{emotion_config[emotion_detected]['name']} ({max_confidence:.0%})",
                                (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (255, 255, 255), 2)

        cv2.imshow('Deteccion de Emociones', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            anger_alarm_playing = False
            stop_anger_alarm_now()
            stop_event.set()
            break

    cap.release()
    cv2.destroyAllWindows()
    anger_alarm_playing = False
    stop_anger_alarm_now()


thread = threading.Thread(target=emotion_detection_thread, daemon=True)
thread.start()
thread.join()


Ventana abierta. Pulsa 'q' para salir.

